# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [26]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [27]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [29]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [30]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [31]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [32]:

# Create Table 

query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(session_id INT, item_in_session INT, artist_name VARCHAR, song_title text, song_duration DECIMAL, PRIMARY KEY ((session_id), item_in_session))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)  
    
           

In [33]:
# Load the data from the CSV file and insert into the Cassandra table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        # Insert into the table
        query = "INSERT INTO music_library (session_id, item_in_session, artist_name, song_title, song_duration) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        # Properly assign the values from CSV to the table attributes
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as exep:
            print(exep)

#### Do a SELECT to verify that the data have been inserted into each table

In [103]:
select_query = "SELECT artist_name, song_title, song_duration FROM music_library WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.artist_name, row.song_title, row.song_duration)

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [36]:
# The PRIMARY KEY is chosen to be ((user_id, sessionId), item_in_session) and PARTITION KEY to be (user_id, sessionId)
# We are querying on user_id and session_id and we want the results to be sorted in order of itemInSession
# Therefore, using (user_id, sessionId) as PARTITION KEY
# As output needs to be sorted in item_in_session, it is chosen to be a clustering column and is added in the PRIMARY KEY 

# Create table
query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(user_id INT, session_id INT, item_in_session INT, artist_name VARCHAR, song_title TEXT, first_name VARCHAR, last_name VARCHAR, \
                    PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)
                    

In [37]:
#Load the data from the csv and insert into the Cassandra table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        # Insert into the table
        query = "INSERT INTO artist_library (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Properly assign the values from CSV to the table attributes
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as exep:
            print(exep)

In [38]:
# SELECT statement to verify whether the data was properly entered into the table
select_query = "SELECT artist_name, song_title, first_name, last_name FROM artist_library WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3 - Retrieve the user name (first and last) in the music app history who listened to a particular song

In [95]:
# The PRIMARY KEY is chosen to be ((song_title), userId) and PARTITION KEY to be song_title
# Using only song_title as PRIMARY KEY will overwrite duplicates as Cassandra does not allow duplicate entries.
# We are querying on the song_title and hence it should be the PARTITION KEY
# Output needs to be sorted in itemInSession, it is chosen to be a clustering column and is added in the PRIMARY KEY 

# Create table
query = "CREATE TABLE IF NOT EXISTS play_history "
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as exep:
    print(exep)
    


In [96]:
#Used a data frame to read the csv file and insert as csv reader was not inserting the complete data

df = pd.read_csv(file)
for row in df.itertuples():
    query = "INSERT INTO play_history (song_title, user_id, first_name, last_name)"
    query = query + " VALUES (%s, %s, %s, %s)"

    try:
        session.execute(query, (row.song, row.userId, row.firstName, row.lastName))
    except Exception as e:
        print(e)

In [99]:
# SELECT statement to verify whether the data was properly entered into the table
select_query = "SELECT first_name, last_name FROM play_history where song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
   #print(row.count) to check for count of rows
   print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [101]:
# Drop tables
query1 = "DROP TABLE music_library"
query2 = "DROP TABLE artist_library"
query3 = "DROP TABLE play_history"

try:
    session.execute(query1)
    session.execute(query2)
    session.execute(query3)
except Exception as exep:
    print(exep)

### Close the session and cluster connection¶

In [102]:
session.shutdown()
cluster.shutdown()